In [ ]:
#default_exp service.filesystem

# Service Filesystem

> Service implementation that stores data in the local filesystem.

In [ ]:
#export
import json,time,datetime,re
from pathlib import Path

In [ ]:
from contextlib import contextmanager
import tempfile, shutil

In [ ]:
@contextmanager
def test_resources():
    temp_path = Path(tempfile.mkdtemp())
    try:
        cwd = get_ipython().run_line_magic('pwd', '')
        print('cwd',cwd)
        print('temp_path',temp_path)
        yield temp_path
    finally:
        shutil.rmtree(temp_path)

In [ ]:
#export
def posts_list_to_dict(posts,key='id'):
    "Convert a list of dictionaries to a dictionary of dictionaries"
    return {post[key]:post for post in posts}

In [ ]:
posts=[dict(id=0,tag='a'),dict(id=-1,tag='B')]
expected={0:posts[0],-1:posts[1]}
assert expected==posts_list_to_dict(posts)

In [ ]:
#export
def migrate(data_dir,output_dir=None):
    posts_file_re=re.compile(r'posts-(?:\d{13}).json')
    data_dir=Path(data_dir)
    output_dir=data_dir if output_dir is None else Path(output_dir)
    for f_name in data_dir.iterdir():
        if not posts_file_re.fullmatch(f_name.name): continue
        with open(f_name) as f: posts = json.load(f)
        if not posts: continue
        print('migrating',f_name,'to',output_dir)
        if not 'status' in posts[0]:
            for post in posts:
                post['last_updated']=post['created']
                post['status']=50 if post['is_deleted']==0 else 20
                del post['is_deleted']
        # subsequent migrations might do something like
        # if not 'other_key' in posts[0]: ...
        with open(output_dir/f_name.name,'w') as f: json.dump(posts,f)

In [ ]:
with test_resources() as temp_path:
    migrate('test/original-format',temp_path)
    with open(Path(temp_path)/'posts-1614707380557.json') as f: posts = json.load(f)
    assert posts[0]=={
        "id": 1614810438150,
        "author_id": 1614707380557,
        "title": "plan: same as yesterday :-(",
        "body": "",
        "created": "2021-03-03 22:27:18",
        "last_updated": "2021-03-03 22:27:18",
        "status": 50}
    assert posts[2]=={
        "id": 1614276006392,
        "author_id": 1614707380557,
        "title": "tt",
        "body": "130",
        "created": "2021-02-25 18:00:06",
        "last_updated": "2021-02-25 18:00:06",
        "status": 20}

cwd C:\Users\Butterp\github\pete88b\web_journal
temp_path C:\Users\Butterp\AppData\Local\Temp\tmp6i9zna2u
migrating test\original-format\posts-1614707380557.json to C:\Users\Butterp\AppData\Local\Temp\tmp6i9zna2u


In [ ]:
#export
class ServiceFilesystem:
    # TODO: DRY
    def __init__(self,data_dir):
        self.data_dir=Path(data_dir)
        self.data_dir.mkdir(parents=True,exist_ok=True)
        
    def read_user_by_id(self,id): 
        if (self.data_dir/'users.json').is_file():
            with open(self.data_dir/'users.json') as f: 
                for user in json.load(f):
                    if user['id']==id: return user
        return None
    
    def read_user_by_username(self,username): 
        if (self.data_dir/'users.json').is_file():
            with open(self.data_dir/'users.json') as f: 
                for user in json.load(f):
                    if user['username']==username: return user
        return None
    
    def create_user(self,username,password): 
        users=[]
        if (self.data_dir/'users.json').is_file():
            with open(self.data_dir/'users.json') as f: users=json.load(f)
        id=round(time.time()*1000)
        users.append(dict(id=id,username=username,password=password))
        with open(self.data_dir/'users.json','w') as f: json.dump(users,f)
        return id
    
    def _add_username(self,post):
        # TODO: check how slow this is ...
        user=self.read_user_by_id(post['author_id'])
        post['username']='Unknown user' if user is None else user['username'] 
        return post
    
    def _posts(self,author_id):
        if (self.data_dir/f'posts-{author_id}.json').is_file():
            with open(self.data_dir/f'posts-{author_id}.json') as f: 
                return json.load(f)
        return []
    
    def read_posts_by_author_id(self,author_id): 
        return [self._add_username(p) for p in self._posts(author_id) if p['status']>30]
        
    def read_post_by_id(self,author_id,id): 
        for post in self._posts(author_id):
            if post['id']==id: return self._add_username(post)
        return None
    
    def _now(self):
        return datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    
    def create_post(self,author_id,title,body):
        posts=self._posts(author_id)
        id,_now=round(time.time()*1000),self._now()
        posts.insert(0,dict(id=id,author_id=author_id,title=title,body=body,
                            created=_now,last_updated=_now,status=50))
        with open(self.data_dir/f'posts-{author_id}.json','w') as f: json.dump(posts,f)
        return id
    
    def update_post_by_id(self,author_id,id,keys,values):
        posts=self._posts(author_id)
        for post in posts:
            if post['id']==id: 
                for key,value in zip(keys,values): post[key]=value
                post['last_updated']=self._now()
                with open(self.data_dir/f'posts-{author_id}.json','w') as f: json.dump(posts,f)
                return post
        return None
        
    def prepare_posts_file_by_author_id(self,author_id):
        if (self.data_dir/f'posts-{author_id}.json').is_file():
            return self.data_dir,f'posts-{author_id}.json'
        return None,None
    
    def upload_posts_from_file(self,author_id,file):
        # TODO: handle non-json formats
        with open(file) as f: 
            posts=json.load(f)
            for post in posts: post['author_id']=author_id
            posts=posts_list_to_dict(posts)
        posts.update(posts_list_to_dict(self._posts(author_id)))
        posts=sorted(posts.values(), key=lambda post: post['id'], reverse=True)
        with open(self.data_dir/f'posts-{author_id}.json','w') as f: json.dump(posts,f)

In [ ]:
# test user functions
with test_resources() as temp_path:
    service=ServiceFilesystem(temp_path)
    assert service.read_user_by_id(1234) is None
    assert service.read_user_by_username('test.user') is None
    user_id=service.create_user('test.user','badPassword')
    expected_user=dict(id=user_id,username='test.user',password='badPassword')
    assert service.read_user_by_id(user_id)==expected_user
    assert service.read_user_by_username('test.user')==expected_user

In [ ]:
with test_resources() as temp_path:
    service=ServiceFilesystem(temp_path)
    post=service._add_username(dict(author_id=123))
    assert 'Unknown user'==post['username']

In [ ]:
def _compare_post(expected,actal):
    "Check values match for all keys in `expected`, which might not be all keys in `actual`"
    for k in expected.keys(): assert expected[k]==actal[k], f'{k}: expected {expected[k]} but found {actal[k]}'

In [ ]:
# test post functions
with test_resources() as temp_path:
    service=ServiceFilesystem(temp_path)
    user_id=service.create_user('test.user','badPassword')
    assert service.read_posts_by_author_id(123)==[]
    assert service.read_posts_by_author_id(user_id)==[]
    assert service.read_post_by_id(user_id,123) is None
    for i in range(3): service.create_post(user_id,f'title{i}','body')
    post_id=service.create_post(user_id,'title','body')
    for i in range(3): service.create_post(user_id,f'title{i}2','body')
    # don't add created to `expected_post` as we don't know what it's value will be
    expected_post=dict(id=post_id,author_id=user_id,title='title',body='body',username='test.user',status=50)
    posts=service.read_posts_by_author_id(user_id)
    assert len(posts)==7
    _compare_post(expected_post,posts[3])
    assert isinstance(posts[3]['created'],str)
    post=service.read_post_by_id(user_id,post_id)
    assert post==posts[3]
    assert post['status']==50
    assert post!=service.update_post_by_id(user_id,post_id,['status'],[20])
    # test prep download
    directory,filename=service.prepare_posts_file_by_author_id(user_id)
    assert isinstance(directory,Path)
    assert isinstance(filename,str)
    assert filename==f'posts-{user_id}.json'
    assert (directory/filename).is_file()

In [ ]:
# test upload
with test_resources() as temp_path:
    service=ServiceFilesystem(temp_path)
    user_id=service.create_user('test.user','badPassword')
    for i in range(3): service.create_post(user_id,f'title{i}','body')
    post_id=service.create_post(user_id,'title','body')
    for i in range(3): service.create_post(user_id,f'title{i}2','body')
    filename=f'posts-{user_id}.json'
    shutil.copyfile(temp_path/filename,temp_path/'posts2upload')
    posts=service.read_posts_by_author_id(user_id)
    service.upload_posts_from_file(user_id,temp_path/'posts2upload') # makes no difference
    actual_posts=service.read_posts_by_author_id(user_id)
    assert posts==actual_posts, f'expected={posts}\nactual={actual_posts}'
    [service.update_post_by_id(user_id,post['id'],['status'],[20]) for post in posts]
    service.upload_posts_from_file(user_id,temp_path/'posts2upload')
    assert []==service.read_posts_by_author_id(user_id) # they are all still deleted
    (temp_path/filename).unlink()
    service.upload_posts_from_file(user_id,temp_path/'posts2upload')
    actual_posts=service.read_posts_by_author_id(user_id)
    assert posts==actual_posts, f'expected={posts}\nactual={actual_posts}'

In [ ]:
# expected=[{'id': 1615194300275, 'author_id': 1615194300273, 'title': 'title22', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}, {'id': 1615194300275, 'author_id': 1615194300273, 'title': 'title12', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}, {'id': 1615194300274, 'author_id': 1615194300273, 'title': 'title02', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}, {'id': 1615194300274, 'author_id': 1615194300273, 'title': 'title', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}, {'id': 1615194300274, 'author_id': 1615194300273, 'title': 'title2', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}, {'id': 1615194300274, 'author_id': 1615194300273, 'title': 'title1', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}, {'id': 1615194300273, 'author_id': 1615194300273, 'title': 'title0', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}]
# found=[{'id': 1615194300275, 'author_id': 1615194300273, 'title': 'title12', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}, {'id': 1615194300274, 'author_id': 1615194300273, 'title': 'title1', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}, {'id': 1615194300273, 'author_id': 1615194300273, 'title': 'title0', 'body': 'body', 'created': '2021-03-08 09:05:00', 'last_updated': '2021-03-08 09:05:00', 'status': 50, 'username': 'test.user'}]
# for i in range(len(expected)):
#     _compare_post(expected[i],found[i])

AssertionError: title: expected title22 but found title12

In [ ]:
# test delete
with test_resources() as temp_path:
    service=ServiceFilesystem(temp_path)
    user_id=service.create_user('test.user','badPassword')
    for i in range(3): service.create_post(user_id,f'title{i}','body')
    post_id=service.create_post(user_id,'title','body')
    for i in range(3): service.create_post(user_id,f'title{i}2','body')
    post=service.read_post_by_id(user_id,post_id)
    assert post['created']==post['last_updated']
    assert post['status']==50
    time.sleep(1) # wait 1s so that last_updated is different
    assert post!=service.update_post_by_id(user_id,post_id,['status'],[20])
    post=service.read_post_by_id(user_id,post_id)
    assert post['status']==20
    assert post['created']!=post['last_updated']
    # deleted posts are readable by ID ...
    expected_post=dict(id=post_id,author_id=user_id,title='title',body='body',username='test.user',status=20)
    _compare_post(expected_post,service.read_post_by_id(user_id,post_id))
    # but are not returned when reading all posts by author
    assert len(service.read_posts_by_author_id(user_id))==6

In [ ]:
# test update
with test_resources() as temp_path:
    service=ServiceFilesystem(temp_path)
    user_id=service.create_user('test.user','badPassword')
    for i in range(3): service.create_post(user_id,f'title{i}','body')
    post_id=service.create_post(user_id,'title','body')
    for i in range(3): service.create_post(user_id,f'title{i}2','body')
    post=service.read_post_by_id(user_id,post_id)
    assert post['created']==post['last_updated']
    expected_post=dict(id=post_id,author_id=user_id,title='title',body='body',username='test.user',status=50)
    _compare_post(expected_post,post)
    time.sleep(1) # wait 1s so that last_updated is different
    updated_post=service.update_post_by_id(user_id,post_id,['title','body'],['new title','new-body'])
    expected_post['title']='new title'
    expected_post['body']='new-body'
    post=service.read_post_by_id(user_id,post_id)
    assert post['created']!=post['last_updated']
    _compare_post(expected_post,post)

In [ ]:
#export
def before_request(app):
    return ServiceFilesystem(app.config['DATA_DIR'])

In [ ]:
#export
def after_request(app,service):
    pass

In [ ]:
#export
def init_service(app):
    print('service.filesystem.init_service')

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()